# One-vs-Rest (OvR) の実装

このノートブックでは、One-vs-Restアプローチを使用した多クラス分類を実装します。

## 学習目標
- One-vs-Restアプローチの理解
- 複数の二値分類器の組み合わせ
- 多クラス分類の評価
- 決定境界の可視化


In [ ]:
# 必要なライブラリのインポート
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import make_classification, load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import (accuracy_score, classification_report, 
                           confusion_matrix, roc_curve, auc)
import warnings
warnings.filterwarnings('ignore')

# 日本語フォントの設定
plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['figure.figsize'] = (12, 8)
sns.set_style("whitegrid")


## 1. データの準備

多クラス分類用のデータセットを生成します。


In [ ]:
# 多クラス分類用のデータセットを生成
X, y = make_classification(
    n_samples=1000,
    n_features=2,
    n_redundant=0,
    n_informative=2,
    n_clusters_per_class=1,
    n_classes=3,
    random_state=42
)

# データの可視化
plt.figure(figsize=(10, 6))
colors = ['red', 'blue', 'green']
labels = ['Class 0', 'Class 1', 'Class 2']

for i in range(3):
    plt.scatter(X[y == i, 0], X[y == i, 1], c=colors[i], label=labels[i], alpha=0.7)

plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.title('Multi-class Classification Dataset')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print(f"データセットの形状: {X.shape}")
print(f"クラスの分布: {np.bincount(y)}")
print(f"クラス数: {len(np.unique(y))}")


In [ ]:
# データの分割と標準化
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# 特徴量の標準化
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("=== データ分割の結果 ===")
print(f"訓練データの形状: {X_train_scaled.shape}")
print(f"テストデータの形状: {X_test_scaled.shape}")
print(f"訓練データのクラス分布: {np.bincount(y_train)}")
print(f"テストデータのクラス分布: {np.bincount(y_test)}")


## 2. One-vs-Restの実装

### 2.1 手動実装


In [ ]:
class OneVsRestClassifier:
    """
    One-vs-Rest分類器の手動実装
    """
    
    def __init__(self, base_classifier):
        """
        パラメータの初期化
        
        Parameters:
        base_classifier: 基底となる二値分類器
        """
        self.base_classifier = base_classifier
        self.classifiers = []
        self.classes = None
    
    def fit(self, X, y):
        """
        モデルの訓練
        
        Parameters:
        X: 特徴量
        y: ラベル
        """
        self.classes = np.unique(y)
        self.classifiers = []
        
        # 各クラスに対して二値分類器を訓練
        for class_label in self.classes:
            # 現在のクラス vs その他のクラスで二値分類
            y_binary = (y == class_label).astype(int)
            
            # 分類器をコピーして訓練
            classifier = type(self.base_classifier)(**self.base_classifier.get_params())
            classifier.fit(X, y_binary)
            self.classifiers.append(classifier)
        
        return self
    
    def predict_proba(self, X):
        """
        確率の予測
        
        Parameters:
        X: 特徴量
        
        Returns:
        probabilities: 予測確率
        """
        probabilities = []
        
        for classifier in self.classifiers:
            # 各分類器の確率を取得
            prob = classifier.predict_proba(X)[:, 1]
            probabilities.append(prob)
        
        # 確率の正規化
        probabilities = np.array(probabilities).T
        probabilities = probabilities / probabilities.sum(axis=1, keepdims=True)
        
        return probabilities
    
    def predict(self, X):
        """
        クラスの予測
        
        Parameters:
        X: 特徴量
        
        Returns:
        predictions: 予測クラス
        """
        probabilities = self.predict_proba(X)
        return self.classes[np.argmax(probabilities, axis=1)]

# 手動実装のテスト
base_classifier = LogisticRegression(random_state=42, max_iter=1000)
ovr_manual = OneVsRestClassifier(base_classifier)
ovr_manual.fit(X_train_scaled, y_train)

# 予測
y_pred_manual = ovr_manual.predict(X_test_scaled)
y_pred_proba_manual = ovr_manual.predict_proba(X_test_scaled)

print("=== 手動実装の結果 ===")
print(f"予測クラス: {y_pred_manual[:10]}")
print(f"予測確率の形状: {y_pred_proba_manual.shape}")
print(f"予測確率の合計: {y_pred_proba_manual.sum(axis=1)[:5]}")


### 2.2 scikit-learnの実装


In [ ]:
# scikit-learnのOneVsRestClassifier
ovr_sklearn = OneVsRestClassifier(LogisticRegression(random_state=42, max_iter=1000))
ovr_sklearn.fit(X_train_scaled, y_train)

# 予測
y_pred_sklearn = ovr_sklearn.predict(X_test_scaled)
y_pred_proba_sklearn = ovr_sklearn.predict_proba(X_test_scaled)

print("=== scikit-learnの実装結果 ===")
print(f"予測クラス: {y_pred_sklearn[:10]}")
print(f"予測確率の形状: {y_pred_proba_sklearn.shape}")
print(f"予測確率の合計: {y_pred_proba_sklearn.sum(axis=1)[:5]}")

# 手動実装とscikit-learnの比較
print(f"\n=== 実装の比較 ===")
print(f"手動実装の精度: {accuracy_score(y_test, y_pred_manual):.4f}")
print(f"scikit-learnの精度: {accuracy_score(y_test, y_pred_sklearn):.4f}")
print(f"予測の一致率: {np.mean(y_pred_manual == y_pred_sklearn):.4f}")


## 3. 決定境界の可視化


In [ ]:
def plot_decision_boundary(model, X, y, title="Decision Boundary"):
    """
    決定境界の可視化
    
    Parameters:
    model: 訓練済みモデル
    X: 特徴量
    y: ラベル
    title: グラフのタイトル
    """
    # メッシュグリッドの作成
    h = 0.01
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))
    
    # 予測
    Z = model.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    
    # プロット
    plt.figure(figsize=(12, 8))
    plt.contourf(xx, yy, Z, levels=2, alpha=0.8, colors=['red', 'blue', 'green'])
    
    # データポイントのプロット
    colors = ['red', 'blue', 'green']
    labels = ['Class 0', 'Class 1', 'Class 2']
    
    for i in range(3):
        plt.scatter(X[y == i, 0], X[y == i, 1], c=colors[i], label=labels[i], 
                   alpha=0.7, edgecolors='black', linewidth=1)
    
    plt.xlabel('Feature 1')
    plt.ylabel('Feature 2')
    plt.title(title)
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()

# 決定境界の可視化
plot_decision_boundary(ovr_sklearn, X_train_scaled, y_train, "One-vs-Rest Decision Boundary")


## 4. モデルの評価


In [ ]:
# 精度の計算
accuracy = accuracy_score(y_test, y_pred_sklearn)

print("=== モデルの評価結果 ===")
print(f"Accuracy: {accuracy:.4f}")

# 混同行列の可視化
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
cm = confusion_matrix(y_test, y_pred_sklearn)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Class 0', 'Class 1', 'Class 2'], 
            yticklabels=['Class 0', 'Class 1', 'Class 2'])
plt.title('Confusion Matrix')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')

# 分類レポート
plt.subplot(1, 2, 2)
plt.text(0.1, 0.9, 'Classification Report:', fontsize=14, fontweight='bold', transform=plt.gca().transAxes)
plt.text(0.1, 0.8, classification_report(y_test, y_pred_sklearn, target_names=['Class 0', 'Class 1', 'Class 2']), 
         fontsize=10, transform=plt.gca().transAxes, verticalalignment='top')
plt.axis('off')
plt.title('Classification Report')

plt.tight_layout()
plt.show()

# 各クラスのROC曲線
plt.figure(figsize=(12, 8))

for i in range(3):
    # クラスiの二値分類として扱う
    y_binary = (y_test == i).astype(int)
    y_proba = y_pred_proba_sklearn[:, i]
    
    fpr, tpr, _ = roc_curve(y_binary, y_proba)
    roc_auc = auc(fpr, tpr)
    
    plt.subplot(2, 2, i+1)
    plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.2f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(f'ROC Curve for Class {i}')
    plt.legend(loc="lower right")
    plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


## 5. 演習問題

### 演習1: クラス数の影響
異なるクラス数（4, 5, 6クラス）でデータセットを生成し、One-vs-Restの性能を比較してみましょう。

### 演習2: 基底分類器の比較
異なる基底分類器（SVM、決定木など）を使用して、One-vs-Restの性能を比較してみましょう。

### 演習3: 確率の正規化
確率の正規化を行わない場合と行う場合の性能の違いを確認してみましょう。


## まとめ

このノートブックでは、One-vs-Restアプローチを使用した多クラス分類を実装しました。

**学習した内容**：
- One-vs-Restアプローチの理論と実装
- 複数の二値分類器の組み合わせ
- 確率の正規化の重要性
- 多クラス分類の評価方法

**重要なポイント**：
- One-vs-Restは実装が簡単で効率的
- 確率の正規化が必要
- クラス不均衡の影響を受けやすい
- 決定境界が複雑になる場合がある

**次のステップ**：
- One-vs-Oneアプローチの学習
- 多項ロジスティック回帰の実装
- より高度な多クラス分類手法の学習
